Merging both the Files

In [5]:
import pprint
import os
import json
import pandas

In [ ]:
with open('Desktop\\Neo4j_data\\all_the_relations.json','r') as f:
    Labels = json.load(f)  

In [ ]:
with open('Desktop\\Neo4j_data\\p01.json') as file:
    file_to_graph = json.load(file)

In [ ]:
def replace_dict(json_obj):
    for a in json_obj:
        for b in Labels:
            if json_obj[a] in Labels[b]['Values']:
                json_obj[a]={"Label" : b, "key": Labels[b]['key'], "value" : json_obj[a]}

In [ ]:
def replace_array(array):
    for i in range(0,len(array)):
        if isinstance(array[i],dict):
            merge_jsons(array[i])
        elif isinstance(array[i],list):
            replace_array(array[i])
        else:
            for b in Labels:
                if array[i] in Labels[b]['Values']:
                    array[i] = {"Label" : b, "key": Labels[b]['key'], "value" : array[i]}
                    break

In [ ]:
def merge_jsons(relations):
    for a in relations:
        if isinstance(file_to_graph['Facts'][a],dict):
            merge_jsons(relations[a])
        elif isinstance(relations[a],list):
            replace_array(relations[a])
        else:
            for b in Labels:
                if relations[a] in Labels[b]['Values']:
                    relations[a]={"Label" : b, "key": Labels[b]['key'], "value" : file_to_graph['Facts'][a]}
                    break

In [ ]:
pprint.pprint(file_to_graph)

In [ ]:
merge_jsons(file_to_graph['Facts'])
with open('Desktop\\Neo4j_data\\merged_file.json', 'w') as output:
    json.dump(file_to_graph,output,indent=8)

In [ ]:
for a in file_to_graph['Facts']:
    pprint.pprint(file_to_graph['Facts'][a])


Creating all the Nodes

In [1]:
from neo4j import GraphDatabase

In [2]:
#connection driver
graphdb = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j","jeet5097"))

In [3]:
session = graphdb.session()

In [6]:
with open("Desktop//Neo4j_data/all_the_relations.json",'r') as data:
    all_the_nodes = json.load(data)

In [7]:
pprint.pprint(all_the_nodes)

{'Case_No': {'Values': ['Case No.316/2014', 'Case No.180/2007'], 'key': 'No'},
 'Currency': {'Values': ['Rs. 22,00,000/-', 'Rs. 2,00,000/-'], 'key': 'Amount'},
 'Date': {'Values': ['14.7.2016',
                     '26.6.2014',
                     '1.12.2008',
                     '10.9.2016',
                     '3.6.2016'],
          'key': 'date'},
 'FIR': {'Values': ['FIR No.38/91'], 'key': 'No'},
 'Name_of_the_file': {'Values': 'p11.txt', 'key': 'Name'},
 'Organisation': {'Values': ['HIGH COURT',
                             'Supreme Court',
                             'Life Insurance Corporation of India'],
                  'key': 'Name'},
 'Person': {'Values': ['SUMITA JARYAL',
                       'ANITA SOOD',
                       'MR. LALIT SEHGAL',
                       'MR. RAJIV SOOD',
                       'AJAY MOHAN GOEL',
                       'Mr. ABC',
                       'Mr. PQR',
                       'Mr. GHI',
                       'Mr. XYZ',
   

In [8]:
for a in all_the_nodes:
    if isinstance(all_the_nodes[a]['Values'],list):
        for i in range(0,len(all_the_nodes[a]['Values'])):
            query='CREATE (:'+a+'{'+all_the_nodes[a]['key']+ ':"' + all_the_nodes[a]['Values'][i] +'"})'
            pprint.pprint(query)
            session.run(query)
    else:
        query='CREATE (:'+a+'{'+all_the_nodes[a]['key']+ ':"' + all_the_nodes[a]['Values'] +'"})'
        pprint.pprint(query)
        session.run(query)

'CREATE (:Name_of_the_file{Name:"p11.txt"})'
'CREATE (:Case_No{No:"Case No.316/2014"})'
'CREATE (:Case_No{No:"Case No.180/2007"})'
'CREATE (:Currency{Amount:"Rs. 22,00,000/-"})'
'CREATE (:Currency{Amount:"Rs. 2,00,000/-"})'
'CREATE (:Date{date:"14.7.2016"})'
'CREATE (:Date{date:"26.6.2014"})'
'CREATE (:Date{date:"1.12.2008"})'
'CREATE (:Date{date:"10.9.2016"})'
'CREATE (:Date{date:"3.6.2016"})'
'CREATE (:Organisation{Name:"HIGH COURT"})'
'CREATE (:Organisation{Name:"Supreme Court"})'
'CREATE (:Organisation{Name:"Life Insurance Corporation of India"})'
'CREATE (:Person{Name:"SUMITA JARYAL"})'
'CREATE (:Person{Name:"ANITA SOOD"})'
'CREATE (:Person{Name:"MR. LALIT SEHGAL"})'
'CREATE (:Person{Name:"MR. RAJIV SOOD"})'
'CREATE (:Person{Name:"AJAY MOHAN GOEL"})'
'CREATE (:Person{Name:"Mr. ABC"})'
'CREATE (:Person{Name:"Mr. PQR"})'
'CREATE (:Person{Name:"Mr. GHI"})'
'CREATE (:Person{Name:"Mr. XYZ"})'
'CREATE (:Person{Name:"Mr. JKL"})'
'CREATE (:Person{Name:"Sanjay Kumar"})'
'CREATE (:Place{Nam

Creating Relations

In [9]:
with open("Desktop/Neo4j_data/merged_file.json",'r') as rel:
    relations_to_be_created = json.load(rel)

In [10]:
pprint.pprint(relations_to_be_created)

{'Facts': {'Acccused': {'Label': 'Person', 'key': 'Name', 'value': 'Mr. XYZ'},
           'Appellant': {'Label': 'Person', 'key': 'Name', 'value': 'Mr. ABC'},
           'Attorneys': {'Appellant': {'Label': 'Person',
                                       'key': 'Name',
                                       'value': 'Mr. ABC'},
                         'Respondent': {'Label': 'Person',
                                        'key': 'Name',
                                        'value': 'Mr. XYZ'}},
           'Case_No.': {'Label': 'Case_No',
                        'key': 'No',
                        'value': 'Case No.180/2007'},
           'FIR_No': {'Label': 'FIR', 'key': 'No', 'value': 'FIR No.38/91'},
           'Final_Judgment': 'Appeal Dismissed',
           'Judge': {'Label': 'Person', 'key': 'Name', 'value': 'Mr. PQR'},
           'Judgment_date': {'Label': 'Date',
                             'key': 'date',
                             'value': '3.6.2016'},
           'Res

In [11]:
def create_relations_of_list(rel_name,list_obj):
    for i in range(0,len(list_obj)):
        query1 = 'Merge (node1:Case_No{No:"'+relations_to_be_created['Facts']['Case_No.']["value"]+'"})'
        query1 = query1+' Merge(node2:'+list_obj[i]['Label']+'{'+ list_obj[i]['key']+':"'+ list_obj[i]['value']+'"}) '
        query1 = query1+ 'Merge (node1)-[:'+ rel_name +']->(node2)'
        print(query1)
        session.run(query1)

In [12]:
def create_relations_of_json(json_obj):
    for a in json_obj:
        if a != 'Case_No.' :
            if 'Label' in json_obj[a]:
                query1 = 'Merge (node1:Case_No{No:"'+json_obj['Case_No.']["value"]+'"})'
                query1 = query1+' Merge(node2:'+json_obj[a]['Label']+'{'+ json_obj[a]['key']+':"'+ json_obj[a]['value']+'"}) '
                query1 = query1+ 'Merge (node1)-[:'+ a +']->(node2)'
                print(query1)
                session.run(query1)
            elif isinstance(json_obj[a],list):
                create_relations_of_list(a,json_obj[a])
#                 continue

In [13]:
create_relations_of_json(relations_to_be_created['Facts'])

Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:Person{Name:"Mr. PQR"}) Merge (node1)-[:Judge]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:Person{Name:"Mr. XYZ"}) Merge (node1)-[:Acccused]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:Person{Name:"Mr. ABC"}) Merge (node1)-[:victim]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:Person{Name:"Sanjay Kumar"}) Merge (node1)-[:Witnesses]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:Person{Name:"Mr. ABC"}) Merge (node1)-[:Appellant]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:Person{Name:"Mr. XYZ"}) Merge (node1)-[:Respondent]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:FIR{No:"FIR No.38/91"}) Merge (node1)-[:FIR_No]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge(node2:Sections_referred{No:"Section 1"}) Merge (node1)-[:Sections_referred]->(node2)
Merge (node1:Case_No{No:"Case No.180/2007"}) Merge

Creating Relations from the file

In [14]:
def split(s):
     parts = []
     bracket_level = 0
     current = []
     for c in (s + ","):
         if c == "," and bracket_level == 0:
             parts.append("".join(current))
             current = []
         else:
             if c == "{":
                 bracket_level += 1
             elif c == "}":
                 bracket_level -= 1
             current.append(c)
     return parts

In [16]:
f = open('Desktop\\Neo4j_data\\realtions_syntax_to_Create_File.txt', "r")
Nodes = f.readlines()
for i in range(0,len(Nodes)):
    node = Nodes[i]
    node=node.replace('(','')
    node=node.replace(')','')
    props=split(node)
    e1,r1,e2 = props[0],props[1],props[2]
    l1=e1.split('{')[0]
    l2=e2.split('{')[0]
    e1props=l1+'{'+e1.split("\"")[1]+':"'+e1.split("\"")[3]+'"}'
    e2props=l2+'{'+e2.split("\"")[1]+':"'+e2.split("\"")[3]+'"}'
    query2 = 'Merge (E1:'+e1props+') Merge (E2:'+e2props+') Merge (E1)-[:'+r1+']->(E2)'
    session.run(query2)
    print(query2)

Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Person{Name:"Mr. ABC"}) Merge (E1)-[:filed_by]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Person{Name:"Mr. XYZ"}) Merge (E1)-[:responded_by]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Person{Name:"Mr. PQR"}) Merge (E1)-[:judged_by]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Case_No{No:"Case No.234/2011"}) Merge (E1)-[:cited]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Case_No{No:"Case No.12/2004"}) Merge (E1)-[:cited]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Case_No{No:"Case No.117/2006"}) Merge (E1)-[:cited]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Case_No{No:"Case No.23/1990"}) Merge (E1)-[:cited]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Sections_referred{No:"Section 1"}) Merge (E1)-[:referred]->(E2)
Merge (E1:Case_No{No:"Case No.101/2012"}) Merge (E2:Sections_referred{No:"Section 2"}) Merge (E1)-[:referred]->(E2)
Merge (E1:

Merge 2 Databases

In [22]:
session.run("call apoc.export.cypherAll('test2.cypher', {format:'plain'})")

Queries to View, Delete and Create etc

In [ ]:
#View the Graph Query
view_graph='MATCH (n:Person) RETURN where n.Name="Mr. XYZ" n'
ans = session.run(view_graph)
for a in ans:    
    pprint.pprint((a))

In [ ]:
#Deletion of all the nodes and relations
del_ent="MATCH (v) DETACH DELETE v"

In [ ]:
#Deletion of any entity having no relationsips
MATCH (n)
WHERE size((n)--())=0
DELETE (n)

In [ ]:
#Deletion of a Relationship
del_rel = "MATCH (entity1)-[r:rel_name]->(entity2) delete r"

In [ ]:
nodes=session.run(view_graph)

In [ ]:
#creating root nodes
session.run('CREATE (:File{name:"p11.txt"})')
session.run('CREATE (:Case_No{No:"180/2007"})')


In [ ]:
#Create root Relationship
val=session.run('MATCH (p11:File{name:"p11.txt"}), (case180:Case_No{No:"180/2007"}) Create (p11)-[:Case_No] ->(case180)')
print(val)

In [ ]:
#Judgment date
session.run('')

In [ ]:
type(nodes)

In [ ]:
q1 = ('MATCH (:Case_No{No:"180/2007"})-[:FIR_Number]->(n:FIR) return n')
nodes = session.run(q1)
for node in nodes:
#     print(dict(dict(node)))
    print(dict(dict(node)['n'])['No'])

In [ ]:
type(nodes)